<a href="https://colab.research.google.com/github/KavyaPatel83/KavyaPatel83.github.io/blob/main/LEX_and_YACC_Compiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEX and YACC Compiler in Colab

Drawbacks:
* Regular interrupts (Ctrl+D, Ctrl+C) for shell won't work in Colab while inputting for program.
<br>Workaround: Store your inputs in a txt file and pass it to the program.

In [2]:
#@title Install *prerqeuisites* (run this cell first to work on LEX/YACC)
!sudo apt install flex bison

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfl-dev libfl2
Suggested packages:
  bison-doc flex-doc
The following NEW packages will be installed:
  bison flex libfl-dev libfl2
0 upgraded, 4 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,072 kB of archives.
After this operation, 3,667 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 flex amd64 2.6.4-8build2 [307 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 bison amd64 2:3.8.2+dfsg-1build1 [748 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfl2 amd64 2.6.4-8build2 [10.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfl-dev amd64 2.6.4-8build2 [6,236 B]
Fetched 1,072 kB in 1s (1,174 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend can

## Lex only

In [3]:
#@title Writing Lex program
%%writefile program.l

%{
    #include <stdio.h>
    int ctChar=0;
    int ctSpace=0;
    int ctWord=0;
    int ctLine=0;
%}
WORD [^ \t\n,\.:]+
EOL [\n]
BLANK [ ]
%%

{WORD} {ctWord++; ctChar+=yyleng;}
{BLANK} {ctSpace++;}
{EOL} {ctLine++;}
. {ctChar++;}
%%

void main(int argc, char *argv[]){
    if(argc!=2){
        printf("Usage:\n\t./a.out <FILENAME>\n");
        exit(0);
    }

    yyin=fopen(argv[1],"r");
    yylex();

    printf("Word Count: %d\n",ctWord);
    printf("Character Count: %d\n",ctChar);
    printf("Space Count: %d\n",ctSpace);
    printf("Line Count: %d\n",ctLine);
    fclose(yyin);

}

int yywrap(){
    return 1;
}

Writing program.l


if you want to use at txt as an input

In [4]:
%%writefile program.txt

This is a sample file.

Writing program.txt


In [5]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell

lex -l program.l
gcc lex.yy.c
./a.out program.txt

Word Count: 5
Character Count: 18
Space Count: 4
Line Count: 2


## Lex and Yacc combined

In [10]:
#@title Writing YACC program
%%writefile first.y

%{
#include <stdio.h>
#include <stdlib.h>
int yylex();
void yyerror(const char *s);
%}

%token NUMBER

%%
expr:
      /* empty */
    | expr '\n'        { printf("Result: %d\n", $1); }
    | expr '+' term    { $$ = $1 + $3; }
    | expr '-' term    { $$ = $1 - $3; }
    | term             { $$ = $1; }
    ;

term:
      NUMBER           { $$ = $1; }
    ;
%%

void yyerror(const char *s) {
    fprintf(stderr, "Error: %s\n", s);
}

int main() {
    printf("Enter an expression (example: 3+5-2) and press Enter:\n");
    yyparse();
    return 0;
}


Overwriting first.y


In [11]:
#@title Writing Lex program
%%writefile firstlex.l

%{
#include "y.tab.h"
%}

%%
[0-9]+     { yylval = atoi(yytext); return NUMBER; }
[+\n-]     { return yytext[0]; }
[ \t]      { /* ignore spaces and tabs */ }
.          { /* ignore unknown chars */ }
%%

int yywrap() {
    return 1;
}


Overwriting firstlex.l


if you want to use at txt as an input

In [12]:
#23BDS1076 - PATEL KAVYA VIPULKUMAR
%%shell

yacc -d first.y
lex firstlex.l
cc y.tab.c lex.yy.c -ll
./a.out

Enter an expression (example: 3+5-2) and press Enter:
3+5-2
Result: 6


CalledProcessError: Command '
yacc -d first.y
lex firstlex.l
cc y.tab.c lex.yy.c -ll
./a.out
' died with <Signals.SIGINT: 2>.

2nd


In [13]:
%%writefile secondyaac.l

%{
#include "y.tab.h"
%}

%%
[0-9]+     { yylval = atoi(yytext); return NUMBER; }
\n         { return '\n'; }
[ \t]      { /* ignore */ }
.          { /* ignore */ }
%%

int yywrap() { return 1; }


Writing secondyaac.l


In [14]:
%%writefile second.y

%{
#include <stdio.h>
#include <stdlib.h>
int yylex();
void yyerror(const char *s);
%}

%token NUMBER

%%
input:
    NUMBER '\n' {
        if($1 % 2 == 0) printf("%d is Even\n", $1);
        else printf("%d is Odd\n", $1);
    }
    ;
%%

void yyerror(const char *s) { printf("Error: %s\n", s); }

int main() {
    printf("Enter a number:\n");
    yyparse();
    return 0;
}


Writing second.y


In [16]:
#23BDS1076 - PATEL KAVYA VIPULKUMAR
%%shell

yacc -d second.y
lex secondyaac.l
cc y.tab.c lex.yy.c -ll
./a.out

Enter a number:
1076
1076 is Even


CalledProcessError: Command '
yacc -d second.y
lex secondyaac.l
cc y.tab.c lex.yy.c -ll
./a.out
' died with <Signals.SIGINT: 2>.

3rd


In [24]:
%%writefile thirdyaac.l

%{
#include "y.tab.h"
#include <string.h>
char str[100];
%}

%%
[a-zA-Z]+ { strcpy(str, yytext); return WORD; }
\n        { return '\n'; }
.         { /* ignore */ }
%%

int yywrap() { return 1; }


Writing thirdyaac.l


In [25]:
%%writefile third.y

%{
#include <stdio.h>
#include <string.h>
int yylex();
void yyerror(const char *s);
extern char str[100];
%}

%token WORD

%%
input:
    WORD '\n' {
        int len = strlen(str), i, flag = 1;
        for(i=0; i<len/2; i++) {
            if(str[i] != str[len-i-1]) { flag = 0; break; }
        }
        if(flag) printf("%s is a Palindrome\n", str);
        else printf("%s is NOT a Palindrome\n", str);
    }
    ;
%%

void yyerror(const char *s) { printf("Error: %s\n", s); }

int main() {
    printf("Enter a word:\n");
    yyparse();
    return 0;
}


Overwriting third.y


In [26]:
#23BDS1076 - PATEL KAVYA VIPULKUMAR
%%shell

yacc -d third.y
lex thirdyaac.l
cc y.tab.c lex.yy.c -ll
./a.out

Enter a word:
madam
madam is a Palindrome


CalledProcessError: Command '
yacc -d third.y
lex thirdyaac.l
cc y.tab.c lex.yy.c -ll
./a.out
' died with <Signals.SIGINT: 2>.

4

In [27]:
%%writefile fourth.y

%{
#include <stdio.h>
int yylex();
void yyerror(const char *s);
int count = 0;
%}

%token WORD

%%
input:
    /* empty */
    | input WORD { count++; }
    | input '\n' { printf("Word Count: %d\n", count); }
    ;
%%

void yyerror(const char *s) { printf("Error: %s\n", s); }

int main() {
    printf("Enter a sentence:\n");
    yyparse();
    return 0;
}


Writing fourth.y


In [28]:
%%writefile fourthyaac.l

%{
#include "y.tab.h"
%}

%%
[a-zA-Z]+ { return WORD; }
\n        { return '\n'; }
[ \t]     { /* ignore */ }
.         { /* ignore */ }
%%

int yywrap() { return 1; }


Writing fourthyaac.l


In [29]:
#23BDS1076 - PATEL KAVYA VIPULKUMAR
%%shell

yacc -d fourth.y
lex fourthyaac.l
cc y.tab.c lex.yy.c -ll
./a.out

Enter a sentence:
Patel Kavya Vipulkumar
Word Count: 3


CalledProcessError: Command '
yacc -d fourth.y
lex fourthyaac.l
cc y.tab.c lex.yy.c -ll
./a.out
' died with <Signals.SIGINT: 2>.

5

In [30]:
%%writefile fifth.y

%{
#include <stdio.h>
int yylex();
void yyerror(const char *s);
int vcount = 0;
%}

%token VOWEL CONSONANT

%%
input:
    /* empty */
    | input VOWEL     { vcount++; }
    | input CONSONANT { }
    | input '\n'      { printf("Vowel Count: %d\n", vcount); }
    ;
%%

void yyerror(const char *s) { printf("Error: %s\n", s); }

int main() {
    printf("Enter text:\n");
    yyparse();
    return 0;
}


Writing fifth.y


In [31]:
%%writefile fifthyaac.l

%{
#include "y.tab.h"
%}

%%
[aeiouAEIOU] { return VOWEL; }
[a-zA-Z]     { return CONSONANT; }
\n           { return '\n'; }
[ \t]        { /* ignore */ }
.            { /* ignore */ }
%%

int yywrap() { return 1; }


Writing fifthyaac.l


In [ ]:
#23BDS1076 - PATEL KAVYA VIPULKUMAR
%%shell

yacc -d fifth.y
lex fifthyaac.l
cc y.tab.c lex.yy.c -ll
./a.out

Enter text:
Hello World
Vowel Count: 3
